In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import torch
import torchvision
from torchvision import transforms
import torch.nn as nn
import torch.nn.functional as F
from scipy.io import loadmat
from torch.utils.data import DataLoader, Dataset
from torch.autograd import Variable

In [2]:
!wget http://ufldl.stanford.edu/housenumbers/train_32x32.mat
!wget http://ufldl.stanford.edu/housenumbers/test_32x32.mat

--2021-04-27 07:58:19--  http://ufldl.stanford.edu/housenumbers/train_32x32.mat
Resolving ufldl.stanford.edu (ufldl.stanford.edu)... 171.64.68.10
Connecting to ufldl.stanford.edu (ufldl.stanford.edu)|171.64.68.10|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 182040794 (174M) [text/plain]
Saving to: ‘train_32x32.mat’

train_32x32.mat     100%[===================>] 173.61M  55.4MB/s    in 3.1s    

2021-04-27 07:58:22 (55.4 MB/s) - ‘train_32x32.mat’ saved [182040794/182040794]

--2021-04-27 07:58:22--  http://ufldl.stanford.edu/housenumbers/test_32x32.mat
Resolving ufldl.stanford.edu (ufldl.stanford.edu)... 171.64.68.10
Connecting to ufldl.stanford.edu (ufldl.stanford.edu)|171.64.68.10|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 64275384 (61M) [text/plain]
Saving to: ‘test_32x32.mat’

test_32x32.mat      100%[===================>]  61.30M  53.5MB/s    in 1.1s    

2021-04-27 07:58:23 (53.5 MB/s) - ‘test_32x32.mat’ saved [64275384/64

In [3]:
class SVHN_dataset(Dataset):

    def __init__(self, data,transform):
        # Transform data to Torch Tensors
        self.images = torch.tensor(data['X']).permute([3,2,0,1])
        self.labels = torch.tensor(data['y'])
        self.size = self.labels.shape[0]

        # replace label 10 with label 0
        self.labels[self.labels==10] = 0
        # convert to float and normalize images to 0..1 range
        self.images = torch.FloatTensor(self.images/255.)
        self.transform = transform

    def __len__(self):
        return self.size

    def __getitem__(self, idx):
        return self.images[idx], self.labels[idx]

In [4]:
class CNN(nn.Module):
  def __init__(self):
    super(CNN, self).__init__()

    self.layer1 = nn.Sequential(
        nn.Conv2d(3,6,(5,5),padding=2,stride=1),
        nn.BatchNorm2d(6),
        nn.ReLU(),
        nn.MaxPool2d(kernel_size=2, stride=2)
    )
    self.layer2 = nn.Sequential(
        nn.Conv2d(6,12,(3,3),padding=1,stride=1),
        nn.BatchNorm2d(12),
        nn.ReLU(),
        nn.MaxPool2d(kernel_size=2, stride=2)
    )
    self.layer3 = nn.Sequential(
        nn.Conv2d(12,24,(3,3),padding=1,stride=1),
        nn.BatchNorm2d(24),
        nn.ReLU(),
        nn.MaxPool2d(kernel_size=2, stride=2)
    )

    self.fc1 = nn.Linear(in_features=24*4*4, out_features=120)
    self.fc2 = nn.Linear(in_features=120, out_features=60)
    self.out = nn.Linear(in_features=60, out_features=10)
  

  def forward(self, x):
    out = self.layer1(x)
    # print(out.shape)

    out = self.layer2(out)
    # print(out.shape)

    out = self.layer3(out)
    # print(out.shape)

    out = out.reshape(-1, 24 * 4 * 4)
    out = self.fc1(out)
    out = F.relu(out)

    out = self.fc2(out)
    out = F.relu(out)

    out = self.out(out)
    out = F.softmax(out)
    # print(out.shape)

    return out

In [5]:
# initialize weights of CNN layers
def init_weights(m):
  mean = 0.0
  std = 0.001
  if isinstance(m, nn.Conv2d):
    m.weight.data.normal_(mean,std)
    if m.bias is not None:
      nn.init.constant_(m.bias.data, 0)
  elif isinstance(m, nn.BatchNorm2d):
    m.weight.data.normal_(mean,std)
    if m.bias is not None:
      nn.init.constant_(m.bias.data, 0)
  elif isinstance(m, nn.Linear):
    m.weight.data.normal_(mean,std)
    if m.bias is not None:
      nn.init.constant_(m.bias.data, 0)

In [6]:
transform = transforms.Compose(
      [
        transforms.ToTensor(),
        transforms.Normalize([0.5, 0.5, 0.5], [0.5, 0.5, 0.5])
        ]
    )

In [7]:
train = loadmat('train_32x32.mat')
test = loadmat('test_32x32.mat')

In [8]:
print(train['X'].shape)

(32, 32, 3, 73257)


In [9]:
train_SVHNdataset = SVHN_dataset(data = train, transform = transform)
test_SVHNdataset = SVHN_dataset(data = test, transform = transform)

In [10]:
params = {'batch_size': 512,
          'shuffle': True,
          'num_workers': 1,
          'pin_memory':True}

In [11]:
trainSVHN_loader = DataLoader(train_SVHNdataset, **params)
testSVHN_loader = DataLoader(test_SVHNdataset, **params)

In [12]:
model = CNN()
model.apply(init_weights)
device = torch.device("cuda:0" if torch.cuda.is_available() else cpu)
print(device)

cuda:0


In [13]:
model.to(device)

CNN(
  (layer1): Sequential(
    (0): Conv2d(3, 6, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (1): BatchNorm2d(6, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (layer2): Sequential(
    (0): Conv2d(6, 12, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): BatchNorm2d(12, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (layer3): Sequential(
    (0): Conv2d(12, 24, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): BatchNorm2d(24, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (fc1): Linear(in_features=384, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=60, bias=True)


In [14]:
learning_rate = 0.001
lossfunc = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(),lr=learning_rate)

In [44]:
num_epochs = 50
# loss_values = list()
for epoch in range(num_epochs):
    # Training
    training_loss = 0
    model.train()
    for X_train, y_train in trainSVHN_loader:
      # Transfer to GPU
      X_train_tensor = torch.tensor(X_train, dtype = torch.float)
      y_train_tensor = torch.tensor(y_train, dtype = torch.long)

      images, labels = Variable(X_train_tensor).to(device), Variable(y_train_tensor).to(device)

      # model computation
      outputs = model(images)
      target = labels.squeeze()
      loss = lossfunc(outputs, target)
      optimizer.zero_grad()
      loss.backward()
      optimizer.step()

      training_loss += loss.item()
    training_loss /= len(trainSVHN_loader)
    print('Epoch -',epoch, 'training loss - ', training_loss)
print('Finished Training')
      # loss_values.append(loss.item())

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:9: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  if __name__ == '__main__':
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:10: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  # Remove the CWD from sys.path while we load stuff.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:47: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.


Epoch - 0 training loss -  1.716782555811935
Epoch - 1 training loss -  1.7175701384743054
Epoch - 2 training loss -  1.7141869970493846
Epoch - 3 training loss -  1.7148590460419655
Epoch - 4 training loss -  1.7145718153980043
Epoch - 5 training loss -  1.7141468649109204
Epoch - 6 training loss -  1.7146702425347433
Epoch - 7 training loss -  1.7134913197822041
Epoch - 8 training loss -  1.714101054602199
Epoch - 9 training loss -  1.7125893690519862
Epoch - 10 training loss -  1.7143131411737866
Epoch - 11 training loss -  1.713500080837144
Epoch - 12 training loss -  1.713014327817493
Epoch - 13 training loss -  1.7135022911760542
Epoch - 14 training loss -  1.7130928842557802
Epoch - 15 training loss -  1.7128936590419874
Epoch - 16 training loss -  1.711631313794189
Epoch - 17 training loss -  1.7114187106490135
Epoch - 18 training loss -  1.711958047416475
Epoch - 19 training loss -  1.7116016679339938
Epoch - 20 training loss -  1.7124073488844767
Epoch - 21 training loss -  1

In [41]:
model.eval()
with torch.no_grad():
  correct = 0
  batches = 0
  for X_test, y_test in testSVHN_loader:
      X_test_tensor = torch.tensor(X_test, dtype = torch.float)
      y_test_tensor = torch.tensor(y_test, dtype = torch.long)
      images, labels = Variable(X_test_tensor).to(device), Variable(y_test_tensor).to(device)
      outputs = model(images)
      target = labels.squeeze()
      _, predicted = torch.max(outputs.data, 1)
      batches += labels.size(0)
      correct += (predicted == target).sum().item()

  print('Test Accuracy of the model: {} %'.format(100 * correct / batches))

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:7: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  import sys
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:47: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.


Test Accuracy of the model: 72.56069452980947 %
